# K-Means Clustering and K Nearest Neighbors Project

Jump to:
* [K Means clustering model](#kmeans)
* [K Nearest Neighbors model](#kneighbor)

## About the dataset

<img src="https://previews.123rf.com/images/aomeditor/aomeditor1903/aomeditor190300021/122254680-illustrator-of-body-parts-of-penguin.jpg" height='400px' width='400px'>

### <b>Columns in the dataset</b>
<ul>
    <li><b>Species: </b>penguin species (Chinstrap, Adélie, or Gentoo)</li>
    <li><b>Island: </b>island name (Dream, Torgersen, or Biscoe) in the Palmer Archipelago (Antarctica)</li>
    <li><b>culmen_length_mm: </b>culmen length (mm)</li>
    <li><b>culmen_depth_mm: </b>culmen depth (mm)</li>
    <li><b>flipper_length_mm: </b>flipper length (mm)</li>
    <li><b>body_mass_g: </b>body mass (g)</li>
    <li><b>Sex: </b>penguin sex</li>
</ul>

In [ ]:
# import basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
data = pd.read_csv('../input/palmer-archipelago-antarctica-penguin-data/penguins_size.csv')
# Check the data
data.info()

In [ ]:
data.head()

# EDA

In [ ]:
sns.countplot(data['species'],palette='spring');

Our data is not balanced. As data is not big enough so I will not balance it.

In case of unbalanced data we can either up-sample minority class or down-sample majority class. To see an example see my following notebooks:
* [up-sampling-minority-class](https://www.kaggle.com/ayushikaushik/up-sampling-to-tackle-unbalanced-dataset)
* [down-sampling-majority-class](https://www.kaggle.com/ayushikaushik/down-sampling-majority-class-6-classification-algo)

To know cons of imbalanced data and more ways to handle it, read [this](https://elitedatascience.com/imbalanced-classes) article.

In [ ]:
sns.pairplot(data,hue='species');

We can see clusters are easily separable in the cases:
1. culmen_length_mm  vs  culmen_depth_mm ;
2. culmen_length_mm  vs  flipper_length_mm ;
3. culmen_length_mm  vs  body_mass_g.

In [ ]:
print("Let's explore distribution of our data.")
fig,axes=plt.subplots(4,1,figsize=(5,20))
sns.boxplot(x=data.species,y=data.flipper_length_mm,ax=axes[0],palette='summer')
axes[0].set_title("Flipper length distribution",fontsize=20,color='Red')
sns.boxplot(x=data.species,y=data.culmen_length_mm,ax=axes[1],palette='rocket')
axes[1].set_title("Culmen length distribution",fontsize=20,color='Red')
sns.boxplot(x=data.species,y=data.culmen_depth_mm,ax=axes[2],palette='twilight')
axes[2].set_title("Culmen depth distribution",fontsize=20,color='Red')
sns.boxplot(x=data.species,y=data.body_mass_g,ax=axes[3],palette='Set2')
axes[3].set_title("Body mass distribution",fontsize=20,color='Red')
plt.tight_layout();

In [ ]:
print("Mean body mass index distribution")
data.groupby(['species','sex']).mean()['body_mass_g'].round(2)

### Dealing with missing values

Let's check out the percentage of missing values.

In [ ]:
100*data.isnull().sum()/len(data)

Percentage of missing data is very less. Let's impute it with median in numerical features and mode in categorical feature.
Here, I have used .fillna method from pandas library.

Missing values can also be filled using pre-defined functions like SimpleImputer from sklearn.

In [ ]:
data['sex'].fillna(data['sex'].mode()[0],inplace=True)
col_to_be_imputed = ['culmen_length_mm', 'culmen_depth_mm','flipper_length_mm', 'body_mass_g']
for item in col_to_be_imputed:
    data[item].fillna(data[item].mean(),inplace=True)

### Dealing with categorical features

In [ ]:
data.species.value_counts()

In [ ]:
data.island.value_counts()

In [ ]:
data.sex.value_counts()

Oops! Where did this '.' entry came from?

In [ ]:
data[data['sex']=='.']

In [ ]:
data.loc[336,'sex'] = 'FEMALE'

In [ ]:
# Target variable can also be encoded using sklearn.preprocessing.LabelEncoder
data['species']=data['species'].map({'Adelie':0,'Gentoo':1,'Chinstrap':2})

# creating dummy variables for categorical features
dummies = pd.get_dummies(data[['island','sex']],drop_first=True)

### Standardizing feature variables.

In [ ]:
# we do not standardize dummy variables 
df_to_be_scaled = data.drop(['island','sex'],axis=1)
target = df_to_be_scaled.species
df_feat= df_to_be_scaled.drop('species',axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df_feat)
df_scaled = scaler.transform(df_feat)
df_scaled = pd.DataFrame(df_scaled,columns=df_feat.columns[:4])
df_preprocessed = pd.concat([df_scaled,dummies,target],axis=1)
df_preprocessed.head()

## <a id='kmeans'>K-Means Clustering</a>

It is very important to note, we actually have the labels for this data set, but we will NOT use them for the K-Means clustering algorithm, since that is an **unsupervised learning algorithm**. K-means is a centroid-based algorithm, or a distance-based algorithm, where we calculate the distances to assign a point to a cluster. In K-Means, each cluster is associated with a centroid. The main objective of the K-Means algorithm is to minimize the sum of distances between the points and their respective cluster centroid.

K Means Clustering is an unsupervised learning algorithm that tries to cluster data based on their similarity. Unsupervised learning means that there is no outcome to be predicted, and the algorithm just tries to find patterns in the data. In k means clustering, we have to specify the ***number of clusters k*** we want the data to be grouped into.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

kmeans = KMeans(3,init='k-means++')
kmeans.fit(df_preprocessed.drop('species',axis=1))
print(confusion_matrix(df_preprocessed.species,kmeans.labels_))

> Running above cell again and again gives different results each time. While making this notebook, best accuracy wass 95%. After commiting, it can show anything. I don't know how to handle this.

In [ ]:
print(classification_report(df_preprocessed.species,kmeans.labels_))

In [ ]:
f"Accuracy is {np.round(100*accuracy_score(df_preprocessed.species,kmeans.labels_),2)}"

In [ ]:
wcss=[]
for i in range(1,10):
    kmeans = KMeans(i)
    kmeans.fit(df_preprocessed.drop('species',axis=1))
    pred_i = kmeans.labels_
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10,6))
plt.plot(range(1,10),wcss)
plt.ylim([0,1800])
plt.title('The Elbow Method',{'fontsize':20})
plt.xlabel('Number of clusters')
plt.ylabel('Within-cluster Sum of Squares');

We can observe from graph that as k goes from 1 to 3 there is a steep decline in wcss. As k increases further decrease in k becomes linear. So, k=3 is a good choice.

## <a id ='kneighbor'>K-Nearest Neighbours Classification</a>

It is a **supervised learning algorithm** which can be used for both classification and regression predictive problems. However, it is more widely used in classification problems in the industry.

With the given data, KNN can classify new, unlabelled data by analysis of the ***k number of the nearest data points***.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# We need to split data for supervised learning models.
X_train, X_test, y_train, y_test = train_test_split(df_preprocessed.drop('species',axis=1),target,test_size=0.50)

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)
preds_knn = knn.predict(X_test)
print(confusion_matrix(y_test,preds_knn))

In [ ]:
plt.title("K Nearest Neighbors Confusion Matrix")
sns.heatmap(confusion_matrix(y_test,preds_knn),annot=True,cmap="Blues",fmt="d",cbar=False, annot_kws={"size": 24});

In [ ]:
print(classification_report(y_test,preds_knn))

In [ ]:
print(accuracy_score(y_test,preds_knn))

In [ ]:
# print the scores on training and test set

print('Training set score: {:.4f}'.format(knn.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(knn.score(X_test, y_test)))

**Figuring out best value for k**

In [ ]:
error_rate=[]
for i in range(1,10):
    knn=KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i!=y_test))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(1,10),error_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate');

From the graph, we can notice that best vaalue for k is 6.

**Best Model:**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train,y_train)
preds_knn = knn.predict(X_test)
print(confusion_matrix(y_test,preds_knn))
print(classification_report(y_test,preds_knn))

So, we achieved 99% accuracy.